In [2]:
import os, sys
from Bio import SeqIO

In [135]:
sc2 = '/home/ngr/ushare/sccovid/data/processed/MT020880.1.gb'
rhv = '/home/ngr/ushare/sccovid/data/processed/HRV-1A_P1.gb'

In [47]:
# DOES NOT WORK
# def gb2gtf(fname,allowedTypes=set(['gene','CDS','tRNA','tmRNA','rRNA','ncRNA', 'mat_peptide']) ):
#     """Reference: https://github.com/lpryszcz/bin/blob/master/gb2gtf.py
    
#     Rewritten due to many errors
#     """
#     for gb in SeqIO.parse(fname,'gb'):
#         acc=gb.id
#         skipped=0
#         skippedTypes=set()
        
#         for f in gb.features:
#             # process only gene and CDS entries
#             if f.type not in allowedTypes:
#                 skipped+=1
#                 skippedTypes.add(f.type)
#                 continue
            
#             comments = ''
#             if 'locus_tag' in f.qualifiers:
#                 gene_id = transcript_id = f.qualifiers['locus_tag'][0]
#                 comments = 'gene_id "%s"; transcript_id "%s"' % (gene_id,transcript_id)
                
#             elif 'gene' in f.qualifiers:
#                 gene_id = transcript_id = f.qualifiers['gene'][0]
#                 comments = 'gene_id "%s"; transcript_id "%s"' % (gene_id,transcript_id)
                
# #             if not comments:
# #                 sys.stderr.write( "Error: Neither `gene` nor `locus_tag` found for entry: %s\n" % '; '.join( str(f).split('\n') ) )
# #                 continue
                
#             if 'product' in f.qualifiers:
#                 comments += '; protein_id "%s"' % f.qualifiers['product'][0]
#             elif 'protein_id' in f.qualifiers:
#                 comments += '; protein_id "%s"' % f.qualifiers['protein_id'][0]
                
#             if 'db_xref' in f.qualifiers:
#                 for extData in f.qualifiers['db_xref']:
#                     comments += '; db_xref "%s"' % extData
                    
#             if int(f.strand)>0: 
#                 strand = '+'
#             else :
#                 strand = '-'
                
#             # seqname - The name of the sequence. Must be a chromosome or scaffold.
#             # source - The program that generated this feature.
#             # feature - The name of this type of feature. Some examples of standard feature types are "CDS", "start_codon", "stop_codon", and "exon".
#             # start - The starting position of the feature in the sequence. The first base is numbered 1.
#             # end - The ending position of the feature (inclusive).
#             # score - A score between 0 and 1000. If the track line useScore attribute is set to 1 for this annotation data set, the score value will determine the level of gray in which this feature is displayed (higher numbers = darker gray). If there is no score value, enter ".".
#             # strand - Valid entries include '+', '-', or '.' (for don't know/don't care).
#             # frame - If the feature is a coding exon, frame should be a number between 0-2 that represents the reading frame of the first base. If the feature is not a coding exon, the value should be '.'.
#             # comments - gene_id "Em:U62317.C22.6.mRNA"; transcript_id "Em:U62317.C22.6.mRNA"; exon_number 1
            
#             gtf = '%s\t%s\t%s\t%s\t%s\t.\t%s\t.\t%s' % (acc,source,f.type,f.location.start.position+1,f.location.end.position,strand,comments ) #f.frame,
            
#             print(gtf)
# #         print('{}\tSkipped {} entries having types {}'.format(gb.id,skipped,skippedTypes))
                        


In [122]:

def gb2gtf(fname):
    """Download complete GeneBank record to get gb file and feed it here.
    
    NOTE: not sure about \n carriage in gtf line.
    
    Args:
      fname (str): full filepath
    
    Returns:
      saved gtf to same directory as fname
      
    References:
      https://useast.ensembl.org/info/website/upload/gff.html
    """
    print('Converting {} to .gtf'.format(os.path.split(fname)[1]))
    outfile = fname.split('.gb')[0]+'.gtf'
    with open(outfile,'a') as out:
        for gb in SeqIO.parse(fname,'gb'):
            acc = gb.id
            for f in gb.features:
                seqname = '.' # seqqname - name of the chromosome or scaffold; chromosome names can be given with or without the 'chr' prefix. Important note: the seqname must be one used within Ensembl, i.e. a standard chromosome name or an Ensembl identifier such as a scaffold ID, without any additional content such as species or assembly.

                # source isn't well defined, take acc and skip source
                if f.type=='source':
                    source = gb.id
                    continue
                else:
                    # skip source feature, otherwise add it 
                    feature = f.type
                    start = f.location.start+1
                    end = f.location.end
                    score = '.' # a floating point value
                    if f.strand > 0:
                        strand = '+'
                    else:
                        strand = '-'

                    # get attributes
                    attribute = ''
                    if 'codon_start' in f.qualifiers:
                        frame = f.qualifiers['codon_start'][0]
                    else:
                        frame = '.'
                    for i in f.qualifiers:
                        if (i != 'translation') and (i != 'codon_start') :
                            if i=='gene':
                                attribute+='gene_name "%s";' % f.qualifiers[i][0]
                            else:
                                attribute+='%s "%s";' % (i,f.qualifiers[i][0])
                    # add spaces to attribute
                    attribute = attribute.replace(';','; ')

                # add feature to file
                gtf = '%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n' % (seqname,source,feature,start,end,score,strand,frame,attribute)        
                out.write(gtf)
        out.close()
        print('... converted {}'.format(os.path.split(outfile)[1]))

In [128]:
attribute = 'gene_id "%s";transcript_id "%s";gene_name "%s";' % ('E','M','S')


In [132]:
attribute[:-1].replace(';','; ')+';'

'gene_id "E"; transcript_id "M"; gene_name "S";'

In [138]:
for gb in SeqIO.parse(sc2,'gb'):
    for f in gb.features:
        print(f)

type: source
location: [0:29882](+)
qualifiers:
    Key: collection_date, Value: ['25-Jan-2020']
    Key: country, Value: ['USA: WA']
    Key: db_xref, Value: ['taxon:2697049']
    Key: host, Value: ['Homo sapiens']
    Key: isolate, Value: ['2019-nCoV/USA-WA1-A12/2020']
    Key: isolation_source, Value: ['nasopharyngeal swab']
    Key: mol_type, Value: ['genomic RNA']
    Key: note, Value: ["passage details: A12 Vero p1; 5' sequence is the primer sequence used"]
    Key: organism, Value: ['Severe acute respiratory syndrome coronavirus 2']

type: gene
location: [265:21555](+)
qualifiers:
    Key: gene, Value: ['orf1ab']

type: CDS
location: join{[265:13468](+), [13467:21555](+)}
qualifiers:
    Key: codon_start, Value: ['1']
    Key: gene, Value: ['orf1ab']
    Key: product, Value: ['orf1ab polyprotein']
    Key: protein_id, Value: ['QHU79193.1']
    Key: ribosomal_slippage, Value: ['']
    Key: translation, Value: ['MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYV